In [2]:
from transformers import AutoModel,AutoTokenizer
import torch
auto_model = AutoModel.from_pretrained("bert-base-cased",output_hidden_states=True)



print(f"\nmodel class is      : {type(auto_model)}")


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print(f"\nmodel class is      : {type(tokenizer)}")


torch.save(auto_model,"hw2/stud/saved/bert.pth")



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



model class is      : <class 'transformers.models.bert.modeling_bert.BertModel'>

model class is      : <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


## Bert and Tokenizer for English

In [4]:
import os
import json
import logging
import torch
from torch.utils.data import DataLoader,Dataset
import random
from typing import Dict

class SRL(Dataset):
 
    def __init__(self,language,tokenizer,path,args_roles = None,pos_list = None,predicate_dis = None) -> None:
        #train
        #self.path_root = 'data'
        #inference 
        self.path_root = 'hw2/stud/data'
        #self.path_root = 'stud/data'
        self.load_data(language,path)
        if args_roles is None :
            self.args_roles,self.list_broken_id = self.list_arg_roles()
            self.args_roles.append("UNK")
        else : 
            self.args_roles = args_roles
            _,self.list_broken_id = self.list_arg_roles()
        

        if pos_list is None :
            self.pos_list,_ = self.list_pos()
            self.pos_list.append("Nothing")
            self.pos_list.append("UNK")
        else : 
            self.pos_list = pos_list
        


        if predicate_dis is None :
            self.predicate_dis,_ = self.list_predicate_roles()
            self.predicate_dis.append("Nothing")
            self.predicate_dis.append("UNK")
        else : 
            self.predicate_dis = predicate_dis
        
        
        


        self.tokenizer = tokenizer

    def load_data(self,language,mode):
        
        mode = mode+".json"
        path = os.path.join(self.path_root,language,mode)
        data_file = open(path)
       
        data_ = json.load(data_file)

        list_data = []

        for data in data_:
            list_data.append(data_[data])
        

        self.data = list_data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, id : int):

        flag = False
        if id in self.list_broken_id :
            flag = True
            while flag == True:

                rand_id = random.randint(0, len(self.data)-1)
                
                if rand_id in self.list_broken_id :
                    pass
                else :
                    flag = False
                    id = rand_id        


        data = self.pre_processing(self.data[id])
        data = self.processig(data)
        return data
        
    def pre_processing(self, data:dict):
        data_list = []
        for role in data["roles"]:
            dictionary = dict()
            dictionary["words"] = data["words"]
            dictionary["role"] = data["roles"][role]
            dictionary["pre_idx"] = role
            dictionary["pos_tags"] = data["pos_tags"]
            dictionary["predicate_meaning"] = data["predicates"]
            data_list.append(dictionary)    
        return data_list
    
    def processig(self,data_list:list):
        
        for dictionary in data_list:

            #dictionary["words"] = data["words"]
            dictionary["gt_arg_identification"] = self.arg_id(dictionary["role"])
            dictionary["gt_arg_classification"] = self.arg_class(dictionary["role"])
            dictionary["pos_idx"] = self.pos_idx(dictionary["pos_tags"])
            dictionary["predicate_meaning_idx"] = self.predicate_meaning_idx(dictionary["predicate_meaning"])
        
        return data_list
   
    def list_arg_roles(self):
        list_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : roles = element["roles"]
            except : flag = False
            if flag :
                for e in roles:
                    sentence = element["roles"][e]

                    for word in sentence:
                        
                        list_roles.append(word)
                list_roles = list(set(list_roles))
            else : 
                list_broken_id.append(i)
        return list_roles,list_broken_id

    def list_predicate_roles(self):
        list_predicate_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : predicates = element["predicates"]
            except : flag = False
            if flag :
                for pre in predicates:
                    list_predicate_roles.append(pre)
                list_predicate_roles = list(set(list_predicate_roles))
            else : 
                list_broken_id.append(i)
        return list_predicate_roles,list_broken_id

    def list_pos(self):
        list_pos = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : pos = element["pos_tags"]
            except : flag = False
            if flag :
                for e in pos:
                    list_pos.append(e)
                list_pos = list(set(list_pos))
            else : 
                list_broken_id.append(i)
        return list_pos,list_broken_id
  
    def arg_class(self,role:list):
        list_idxs = []
        for element in role:
            try : list_idxs.append(self.args_roles.index(element))
            except : list_idxs.append(self.args_roles.index("UNK"))
        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def arg_id(self,role:dict):
        list_idxs = []
        for element in role:
            if element == "_":
                list_idxs.append(0)
            else :
                list_idxs.append(1)

        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def pos_idx(self,pos_tags:dict):
        list_idxs = []
        list_idxs.append(self.pos_list.index("Nothing"))

        for element in pos_tags:
            try :list_idxs.append(self.pos_list.index(element))
            except :list_idxs.append(self.pos_list.index("UNK"))
        
        list_idxs.append(self.pos_list.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
    
    def predicate_meaning_idx(self,predicate_meaning_tags:dict):
        list_idxs = []
        list_idxs.append(self.predicate_dis.index("Nothing"))

        for element in predicate_meaning_tags:
            try : list_idxs.append(self.predicate_dis.index(element))
            except : list_idxs.append(self.predicate_dis.index("UNK"))
            
        
        list_idxs.append(self.predicate_dis.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
   
    def role_gen(self,sentence):

        base = ["_"]*len(sentence["predicates"])
        roles_dict = dict()
        counter = 0
        for i,item in enumerate(sentence["predicates"]):

            if item != "_":
                base = ["_"]*len(sentence["predicates"])
                sentence["roles"] = 10
                roles_dict[str(i)] = base
                counter += 1
        
        if counter == 0:
            sentence["roles"] = { }
            flag = False
            
                
        else :
            sentence["roles"] = roles_dict
            flag = True

        return sentence,flag
        
    def prepare_batch(self,sentence):

        sentence,flag = self.role_gen(sentence)
        
        if flag :

            data = self.pre_processing(sentence)
            data = self.processig(data)
            data = [data]
            
            
            input = dict() 
            gt = dict()
            batch_sentence = [] 
            
            for period in data:
                for sentence in period :

                    
                
                    #print(len(sentence[0]["words"]))
                    pre_idx = int(sentence["pre_idx"])
                    

                    predicate = sentence["words"][pre_idx]

                    text = " ".join(sentence["words"])
                    tokens: list[str] = text.split()
                    predicate: list[str] = predicate.split()

                    #text = sentence[0]["words"]
                    
                    t = (tokens,predicate)

                    batch_sentence.append(t)
                
                
            
        
        

            batch_output = self.tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
            


            for period in data:

                list_positional_predicate_encoding = []
                list_predicate_index = [] 
                list_pos_index = [] 
                list_arg_gt = []
                list_predicate_meaning_index = []
                list_meaning_predicate_encoding = []

                for sentence in period:
                    #positional_encoding
                    #+2 per il CLS iniziale ad SEP finale
                    sentence_words_lenght =  len(sentence["words"])
                    positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    #+1 per il CLS iniziale
                    pre_idx = int(sentence["pre_idx"])
                    positional_predicate_encoding[:,pre_idx+1] = 1
                    list_positional_predicate_encoding.append(positional_predicate_encoding)
                    #print("positional_prefix_encoding",positional_predicate_encoding)
                    list_predicate_index.append(pre_idx)

                    meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    pre_idx = int(sentence["pre_idx"])
                    #rather then set the flag 0,1 set with class verb
                    meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
                    list_meaning_predicate_encoding.append(meaning_predicate_encoding)
                    

                    pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
                    list_pos_index.append(pos)
                    predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
                    list_predicate_meaning_index.append(predicate_meaning_idxs)


                    arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
                    list_arg_gt.append(arg_gt)


            list_arg_gt = torch.cat(list_arg_gt,dim = 0)
            list_pos_index = torch.cat(list_pos_index,dim = 0)
            list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
            list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
            list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
            gt["arg_gt"] = list_arg_gt
            input["predicate_index"] = list_predicate_index
            input["pos_index"] = list_pos_index.long()
            input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
            input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
            offset = batch_output.pop("offset_mapping")
            input["BERT_input"] = batch_output
            input["positional_encoding"] = list_positional_predicate_encoding.long()
            input["offset_mapping"] = offset
            input["gt"] = gt
        
        else :
            input = sentence






        return input,flag
    
# here we define our collate function
def collate_fn(batch) -> Dict[str, torch.Tensor]:
    #print(batch)
    input = dict() 
    batch_sentence = [] 
    #print(len(batch))
    for period in batch:
        for sentence in period :
        
            #print(len(sentence[0]["words"]))
            pre_idx = int(sentence["pre_idx"])
            

            predicate = sentence["words"][pre_idx]

            text = " ".join(sentence["words"])
            tokens: list[str] = text.split()
            predicate: list[str] = predicate.split()

            #text = sentence[0]["words"]
            
            t = (tokens,predicate)

            batch_sentence.append(t)
            #print(batch_sentence)

    batch_output = tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
    #print(batch_output.keys())


    gt = dict()
    
    


    for period in batch:

        list_positional_predicate_encoding = []
        list_arg_gt = []
        list_predicate_index = [] 
        list_pos_index = [] 
        list_predicate_meaning_index = []
        list_meaning_predicate_encoding = []

        for sentence in period:
            #positional_encoding
            #+2 per il CLS iniziale ad SEP finale
            sentence_words_lenght =  len(sentence["words"])
            positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            #+1 per il CLS iniziale
            pre_idx = int(sentence["pre_idx"])
            positional_predicate_encoding[:,pre_idx+1] = 1
            list_positional_predicate_encoding.append(positional_predicate_encoding)
            #print("positional_prefix_encoding",positional_predicate_encoding)
            list_predicate_index.append(pre_idx)


            meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            pre_idx = int(sentence["pre_idx"])
            #rather then set the flag 0,1 set with class verb
            meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
            list_meaning_predicate_encoding.append(meaning_predicate_encoding)




            pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
            list_pos_index.append(pos)
            predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
            list_predicate_meaning_index.append(predicate_meaning_idxs)






            #note CLS and SEP are discharder after Bi-LSTM, the Classifier takes in input only wokrds hidden state embedding
            arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
            list_arg_gt.append(arg_gt)
        

    list_arg_gt = torch.cat(list_arg_gt,dim = 0)
    list_pos_index = torch.cat(list_pos_index,dim = 0)
    list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
    list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
    list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
    gt["arg_gt"] = list_arg_gt
    input["predicate_index"] = list_predicate_index
    input["pos_index"] = list_pos_index.long()
    input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
    input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
    offset = batch_output.pop("offset_mapping")
    input["BERT_input"] = batch_output
    input["positional_encoding"] = list_positional_predicate_encoding.long()
    input["offset_mapping"] = offset
    input["gt"] = gt

   


    return input



In [5]:
train_dataset = SRL("EN",tokenizer,"train")
#same mapping should be used in both the dataset
dev_dataset = SRL("EN",tokenizer,"dev",train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)



In [6]:
#print(train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)



import json

data = {
    'args_roles' : train_dataset.args_roles,
    "pos_list" : train_dataset.pos_list,
    "predicate_dis" : train_dataset.predicate_dis,
}





with open('hw2/stud/saved/vocabulary.json', 'w') as outfile:
    json.dump(data, outfile)


with open('hw2/stud/saved/vocabulary.json') as json_file:
    data = json.load(json_file)

print(data)
pos_list = data['pos_list']
args_roles = data['args_roles']
predicate_dis = data['predicate_dis']
print(pos_list)
   

{'args_roles': ['recipient', 'experiencer', 'product', 'purpose', 'value', 'patient', 'beneficiary', 'asset', 'theme', 'location', 'time', '_', 'goal', 'agent', 'cause', 'topic', 'extent', 'source', 'co-agent', 'attribute', 'co-patient', 'destination', 'stimulus', 'co-theme', 'result', 'instrument', 'material', 'UNK'], 'pos_list': ['PROPN', 'AUX', 'CCONJ', 'NOUN', 'SYM', 'ADV', 'X', 'NUM', 'INTJ', 'PUNCT', 'DET', 'ADP', 'SCONJ', 'PART', 'PRON', 'ADJ', 'VERB', 'Nothing', 'UNK'], 'predicate_dis': ['ORIENT', 'SPEND-TIME_PASS-TIME', 'INFORM', 'TAKE-INTO-ACCOUNT_CONSIDER', 'REDUCE_DIMINISH', 'DIRECT_AIM_MANEUVER', 'STEAL_DEPRIVE', 'DISCARD', 'VISIT', 'ESTABLISH', 'CANCEL_ELIMINATE', 'DOWNPLAY_HUMILIATE', 'OVERCOME_SURPASS', 'MEASURE_EVALUATE', 'COMBINE_MIX_UNITE', 'GUESS', 'LOWER', 'TOLERATE', 'NEGOTIATE', 'ADJUST_CORRECT', 'BORDER', 'CAUSE-MENTAL-STATE', 'LEAD_GOVERN', 'TEACH', 'ACCOMPANY', 'ABSTAIN_AVOID_REFRAIN', 'REFLECT', 'LEARN', 'GO-FORWARD', 'CAGE_IMPRISON', 'RECORD', 'PUNISH', 'DIS

## Model

In [7]:


embeddings = dict()

embeddings["predicate_flag_embedding_output_dim"] = 32
#defined in initial exploration of the dataset
embeddings["pos_embedding_input_dim"] = 0
embeddings["pos_embedding_output_dim"] = 100
#-------------------------------------------------
embeddings["predicate_embedding_input_dim"] = 0
embeddings["predicate_embedding_output_dim"] = 50
#defined in initial exploration of the dataset
n_classes = 0



bilstm = dict()
bilstm["n_layers"] = 2
bilstm["output_dim"] = 50
dropouts = [0.4,0.3,0.3]

language_portable = True
predicate_meaning = True
pos = True

cfg = dict()
cfg["embeddings"] = embeddings
cfg["n_classes"] = n_classes
cfg["bilstm"] = bilstm
cfg["language_portable"] = language_portable
cfg["dropouts"] = dropouts

In [10]:
#from mmcv import Config
from hw2.stud.arg import Arg_Classifier, Arg_Classifier_from_paper



#cfg = Config.fromfile('/home/francesco/Desktop/nlp2022-hw2-main/hw2/stud/configs/model.py')

cfg["embeddings"]["pos_embedding_input_dim"] = len(train_dataset.pos_list)
cfg["embeddings"]["predicate_embedding_input_dim"] = len(train_dataset.predicate_dis)
cfg["n_classes"] = len(train_dataset.args_roles)


model = Arg_Classifier(cfg).cuda()
#model = Arg_Classifier_from_paper("EN",cfg).cuda()
print(model)

automodel = auto_model.cuda()

Arg_Classifier(
  (bi_lstm_portable): LSTM(132, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (embedding_predicate_flag): Embedding(2, 32, max_norm=True)
  (embedding_predicate): Embedding(305, 50, max_norm=True)
  (embedding_pos): Embedding(19, 100, max_norm=True)
  (bi_lstm): LSTM(950, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout_language_constraint): Dropout(p=0.6, inplace=False)
  (dropout_in_classifier): Dropout(p=0.4, inplace=False)
  (Relu): ReLU()
  (Sigmoid): Sigmoid()
  (linear0): Linear(in_features=300, out_features=700, bias=True)
  (linear1): Linear(in_features=700, out_features=140, bias=True)
  (linear2): Linear(in_features=140, out_features=28, bias=True)
)


In [11]:
def metrics(gold,pred):
    true_positives, false_positives, false_negatives = 0, 0, 0
    null_tag = "_"

    for i,r_g in  enumerate(gold):
        r_p = pred[i]

        if r_g != null_tag and r_p != null_tag:
            true_positives += 1
        elif r_g != null_tag and r_p == null_tag:
            false_negatives += 1
        elif r_g == null_tag and r_p != null_tag:
            false_positives += 1

    a = true_positives + false_positives
    b = true_positives + false_negatives
    if a == 0 and b == 0 :        
        argument_identification = {
            "true_positives": 0,
            "false_positives": 0,
            "false_negatives": 0,
            "precision": 0,
            "recall": 0,
            "f1": 0,
        } 

    else : 
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        argument_identification = {
            "true_positives": true_positives,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }




    true_positives, false_positives, false_negatives = 0, 0, 0
    for i,r_g in  enumerate(gold):
        r_p = pred[i]

        if r_g != null_tag and r_p != null_tag:
            if r_g == r_p:
                true_positives += 1
            else:
                false_positives += 1
                false_negatives += 1
        elif r_g != null_tag and r_p == null_tag:
                false_negatives += 1
        elif r_g == null_tag and r_p != null_tag:
                false_positives += 1


    a = true_positives + false_positives
    b = true_positives + false_negatives
    if a == 0 and b == 0 :
        argument_classification = {
            "true_positives": 0,
            "false_positives": 0,
            "false_negatives": 0,
            "precision": 0,
            "recall": 0,
            "f1": 0,
        } 

    else : 
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * (precision * recall) / (precision + recall)
        argument_classification = {
            "true_positives": true_positives,
            "false_positives": false_positives,
            "false_negatives": false_negatives,
            "precision": precision,
            "recall": recall,
            "f1": f1,
        }


    return argument_identification,argument_classification

"""
from [1,1,8,8,8,8,8,8,8,8,8,8,8,8,2,2,2,8,8,8......]
to from [agent,agent,_,_,_........,]
"""
def mapping_args(g,p,mapping):
    
    
    gt = [mapping[elem] for elem in g]
    predictions = [mapping[elem] for elem in p]


    return gt,predictions


## Training Argument Identification and Classification

In [12]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime



currentDateAndTime = datetime.now()
_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)



#optimizer = torch.optim.Adam(model.parameters(),lr = 0.000005)
optimizer = torch.optim.Adam(model.parameters())


scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()

EPOCHS = 200
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
        
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1)
        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    scheduler.step()

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    print("Epochs n.", epoch)
    print("F1 train:",f1)
    print("F1 avg train:",f1_avg)
    
    avg_train_loss = total_loss/counter
    writer.add_scalar("Loss/train", avg_train_loss, epoch)


    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Train_EN/identification", identification_result["f1"], epoch)
    writer.add_scalar("Train_EN/classification", classification_result["f1"], epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------
    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
    

    writer.add_scalar("Loss/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Eval_EN/identification", identification_result["f1"], epoch)
    writer.add_scalar("Eval_EN/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        print("SAVED :",PATH)
        torch.save(model.state_dict(),PATH)
    

Epochs n. 0
F1 train: [0.04415011 0.         0.         0.         0.         0.17993795
 0.         0.         0.44318018 0.         0.         0.98443584
 0.06132666 0.64229412 0.         0.25796178 0.         0.
 0.         0.         0.         0.         0.         0.
 0.02020202 0.         0.         0.        ]
F1 avg train: 0.9512728906843513
identification {'true_positives': 12671, 'false_positives': 1783, 'false_negatives': 12175, 'precision': 0.8766431437664315, 'recall': 0.5099814859534734, 'f1': 0.6448346055979642}
classification_result {'true_positives': 8021, 'false_positives': 6433, 'false_negatives': 16825, 'precision': 0.5549328905493289, 'recall': 0.32282862432584725, 'f1': 0.4081933842239186}
EPOCHS : 0
F1 eval : [0.30522088 0.         0.         0.         0.         0.1549101
 0.         0.         0.51717902 0.         0.         0.98874176
 0.23419204 0.70358306 0.         0.25       0.         0.
 0.         0.         0.         0.         0.         0.
 0.   

## Language Transfert Learning 

### Fine-Tuning over Structural Information over English dataset


#### Loading English Pretrained Model 

In [13]:
#note that parameter EN is only used for tracking on which dataset was/is trained
model = Arg_Classifier(cfg).cuda()
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

#### Language constained training

In [14]:
model.set_language_constrains()
#model.freeze_parts()

#### Fine-Tuning

In [15]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime



currentDateAndTime = datetime.now()
_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)
_id =  _id +"Language_constained_training"



optimizer = torch.optim.Adam(model.parameters(),lr = 0.00000005)
scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()

EPOCHS = 2
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1).cpu()
        b,n = sample_batched["gt"]["arg_gt"].size()


        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    print("Epochs n.", epoch)
    print("F1 train:",f1_score(g, p, average=None))
    scheduler.step()
    avg_train_loss = total_loss/counter
    writer.add_scalar("Loss/train", avg_train_loss, epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------

    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 eval :",f1_avg)
    

    writer.add_scalar("Loss/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Eval_EN/identification", identification_result["f1"], epoch)
    writer.add_scalar("Eval_EN/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        torch.save(model.state_dict(),PATH)
    



    





    


Epochs n. 0
F1 train: [0.60040568 0.57312253 0.06504065 0.         0.         0.77299011
 0.41314554 0.         0.73040578 0.20740741 0.18181818 0.99051611
 0.47439353 0.80747382 0.         0.60424469 0.15625    0.29916898
 0.17777778 0.32822757 0.11940299 0.26       0.51419032 0.27968338
 0.37821782 0.         0.        ]
EPOCHS : 0
F1 eval : [0.77070064 0.62626263 0.25       0.         0.         0.84567901
 0.61135371 0.         0.79694073 0.22222222 0.         0.99324028
 0.59744991 0.83655982 0.         0.85139319 0.33333333 0.41558442
 0.50980392 0.47191011 0.21052632 0.48192771 0.64220183 0.55670103
 0.48407643 0.         0.        ]
F1 eval : 0.9818441786344059
identification {'true_positives': 4204, 'false_positives': 452, 'false_negatives': 859, 'precision': 0.9029209621993127, 'recall': 0.8303377444203042, 'f1': 0.8651095791748122}
classification_result {'true_positives': 3752, 'false_positives': 904, 'false_negatives': 1311, 'precision': 0.8058419243986255, 'recall': 0.7410

### Spanish

#### New Spanish Dataset
bert-base-multilingual-cased


In [18]:
from transformers import BertTokenizer, BertModel
import os
import json
import logging
import torch
from torch.utils.data import DataLoader,Dataset
import random
from typing import Dict



auto_model = AutoModel.from_pretrained("bert-base-multilingual-cased",output_hidden_states=True)
print(f"\nmodel class is      : {type(auto_model)}")

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
print(f"\nmodel class is      : {type(tokenizer)}")



import os
import json
import logging
import torch
from torch.utils.data import DataLoader,Dataset
import random
from typing import Dict

class SRL(Dataset):
 
    def __init__(self,language,tokenizer,path,args_roles = None,pos_list = None,predicate_dis = None) -> None:
        #train
        #self.path_root = 'data'
        #inference 
        self.path_root = 'hw2/stud/data'
        #self.path_root = 'stud/data'
        self.load_data(language,path)
        if args_roles is None :
            self.args_roles,self.list_broken_id = self.list_arg_roles()
            self.args_roles.append("UNK")
        else : 
            self.args_roles = args_roles
            _,self.list_broken_id = self.list_arg_roles()
        

        if pos_list is None :
            self.pos_list,_ = self.list_pos()
            self.pos_list.append("Nothing")
            self.pos_list.append("UNK")
        else : 
            self.pos_list = pos_list
        


        if predicate_dis is None :
            self.predicate_dis,_ = self.list_predicate_roles()
            self.predicate_dis.append("Nothing")
            self.predicate_dis.append("UNK")
        else : 
            self.predicate_dis = predicate_dis
        
        
        


        self.tokenizer = tokenizer

    def load_data(self,language,mode):
        
        mode = mode+".json"
        path = os.path.join(self.path_root,language,mode)
        data_file = open(path)
       
        data_ = json.load(data_file)

        list_data = []

        for data in data_:
            list_data.append(data_[data])
        

        self.data = list_data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, id : int):

        flag = False
        if id in self.list_broken_id :
            flag = True
            while flag == True:

                rand_id = random.randint(0, len(self.data)-1)
                
                if rand_id in self.list_broken_id :
                    pass
                else :
                    flag = False
                    id = rand_id        


        data = self.pre_processing(self.data[id])
        data = self.processig(data)
        return data
        
    def pre_processing(self, data:dict):
        data_list = []
        for role in data["roles"]:
            dictionary = dict()
            dictionary["words"] = data["words"]
            dictionary["role"] = data["roles"][role]
            dictionary["pre_idx"] = role
            dictionary["pos_tags"] = data["pos_tags"]
            dictionary["predicate_meaning"] = data["predicates"]
            data_list.append(dictionary)    
        return data_list
    
    def processig(self,data_list:list):
        
        for dictionary in data_list:

            #dictionary["words"] = data["words"]
            dictionary["gt_arg_identification"] = self.arg_id(dictionary["role"])
            dictionary["gt_arg_classification"] = self.arg_class(dictionary["role"])
            dictionary["pos_idx"] = self.pos_idx(dictionary["pos_tags"])
            dictionary["predicate_meaning_idx"] = self.predicate_meaning_idx(dictionary["predicate_meaning"])
        
        return data_list
   
    def list_arg_roles(self):
        list_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : roles = element["roles"]
            except : flag = False
            if flag :
                for e in roles:
                    sentence = element["roles"][e]

                    for word in sentence:
                        
                        list_roles.append(word)
                list_roles = list(set(list_roles))
            else : 
                list_broken_id.append(i)
        return list_roles,list_broken_id

    def list_predicate_roles(self):
        list_predicate_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : predicates = element["predicates"]
            except : flag = False
            if flag :
                for pre in predicates:
                    list_predicate_roles.append(pre)
                list_predicate_roles = list(set(list_predicate_roles))
            else : 
                list_broken_id.append(i)
        return list_predicate_roles,list_broken_id

    def list_pos(self):
        list_pos = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : pos = element["pos_tags"]
            except : flag = False
            if flag :
                for e in pos:
                    list_pos.append(e)
                list_pos = list(set(list_pos))
            else : 
                list_broken_id.append(i)
        return list_pos,list_broken_id
  
    def arg_class(self,role:list):
        list_idxs = []
        for element in role:
            try : list_idxs.append(self.args_roles.index(element))
            except : list_idxs.append(self.args_roles.index("UNK"))
        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def arg_id(self,role:dict):
        list_idxs = []
        for element in role:
            if element == "_":
                list_idxs.append(0)
            else :
                list_idxs.append(1)

        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def pos_idx(self,pos_tags:dict):
        list_idxs = []
        list_idxs.append(self.pos_list.index("Nothing"))

        for element in pos_tags:
            try :list_idxs.append(self.pos_list.index(element))
            except :list_idxs.append(self.pos_list.index("UNK"))
        
        list_idxs.append(self.pos_list.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
    
    def predicate_meaning_idx(self,predicate_meaning_tags:dict):
        list_idxs = []
        list_idxs.append(self.predicate_dis.index("Nothing"))

        for element in predicate_meaning_tags:
            try : list_idxs.append(self.predicate_dis.index(element))
            except : list_idxs.append(self.predicate_dis.index("UNK"))
            
        
        list_idxs.append(self.predicate_dis.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
   
    def role_gen(self,sentence):

        base = ["_"]*len(sentence["predicates"])
        roles_dict = dict()
        counter = 0
        for i,item in enumerate(sentence["predicates"]):

            if item != "_":
                base = ["_"]*len(sentence["predicates"])
                sentence["roles"] = 10
                roles_dict[str(i)] = base
                counter += 1
        
        if counter == 0:
            sentence["roles"] = { }
            flag = False
            
                
        else :
            sentence["roles"] = roles_dict
            flag = True

        return sentence,flag
        
    def prepare_batch(self,sentence):

        sentence,flag = self.role_gen(sentence)
        
        if flag :

            data = self.pre_processing(sentence)
            data = self.processig(data)
            data = [data]
            
            
            input = dict() 
            gt = dict()
            batch_sentence = [] 
            
            for period in data:
                for sentence in period :

                    
                
                    #print(len(sentence[0]["words"]))
                    pre_idx = int(sentence["pre_idx"])
                    

                    predicate = sentence["words"][pre_idx]

                    text = " ".join(sentence["words"])
                    tokens: list[str] = text.split()
                    predicate: list[str] = predicate.split()

                    #text = sentence[0]["words"]
                    
                    t = (tokens,predicate)

                    batch_sentence.append(t)
                
                
            
        
        

            batch_output = self.tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
            


            for period in data:

                list_positional_predicate_encoding = []
                list_predicate_index = [] 
                list_pos_index = [] 
                list_arg_gt = []
                list_predicate_meaning_index = []
                list_meaning_predicate_encoding = []

                for sentence in period:
                    #positional_encoding
                    #+2 per il CLS iniziale ad SEP finale
                    sentence_words_lenght =  len(sentence["words"])
                    positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    #+1 per il CLS iniziale
                    pre_idx = int(sentence["pre_idx"])
                    positional_predicate_encoding[:,pre_idx+1] = 1
                    list_positional_predicate_encoding.append(positional_predicate_encoding)
                    #print("positional_prefix_encoding",positional_predicate_encoding)
                    list_predicate_index.append(pre_idx)

                    meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    pre_idx = int(sentence["pre_idx"])
                    #rather then set the flag 0,1 set with class verb
                    meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
                    list_meaning_predicate_encoding.append(meaning_predicate_encoding)
                    

                    pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
                    list_pos_index.append(pos)
                    predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
                    list_predicate_meaning_index.append(predicate_meaning_idxs)


                    arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
                    list_arg_gt.append(arg_gt)


            list_arg_gt = torch.cat(list_arg_gt,dim = 0)
            list_pos_index = torch.cat(list_pos_index,dim = 0)
            list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
            list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
            list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
            gt["arg_gt"] = list_arg_gt
            input["predicate_index"] = list_predicate_index
            input["pos_index"] = list_pos_index.long()
            input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
            input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
            offset = batch_output.pop("offset_mapping")
            input["BERT_input"] = batch_output
            input["positional_encoding"] = list_positional_predicate_encoding.long()
            input["offset_mapping"] = offset
            input["gt"] = gt
        
        else :
            input = sentence






        return input,flag
    
# here we define our collate function
def collate_fn(batch) -> Dict[str, torch.Tensor]:
    #print(batch)
    input = dict() 
    batch_sentence = [] 
    #print(len(batch))
    for period in batch:
        for sentence in period :
        
            #print(len(sentence[0]["words"]))
            pre_idx = int(sentence["pre_idx"])
            

            predicate = sentence["words"][pre_idx]

            text = " ".join(sentence["words"])
            tokens: list[str] = text.split()
            predicate: list[str] = predicate.split()

            #text = sentence[0]["words"]
            
            t = (tokens,predicate)

            batch_sentence.append(t)
            #print(batch_sentence)

    batch_output = tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
    #print(batch_output.keys())


    gt = dict()
    
    


    for period in batch:

        list_positional_predicate_encoding = []
        list_arg_gt = []
        list_predicate_index = [] 
        list_pos_index = [] 
        list_predicate_meaning_index = []
        list_meaning_predicate_encoding = []

        for sentence in period:
            #positional_encoding
            #+2 per il CLS iniziale ad SEP finale
            sentence_words_lenght =  len(sentence["words"])
            positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            #+1 per il CLS iniziale
            pre_idx = int(sentence["pre_idx"])
            positional_predicate_encoding[:,pre_idx+1] = 1
            list_positional_predicate_encoding.append(positional_predicate_encoding)
            #print("positional_prefix_encoding",positional_predicate_encoding)
            list_predicate_index.append(pre_idx)


            meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            pre_idx = int(sentence["pre_idx"])
            #rather then set the flag 0,1 set with class verb
            meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
            list_meaning_predicate_encoding.append(meaning_predicate_encoding)




            pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
            list_pos_index.append(pos)
            predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
            list_predicate_meaning_index.append(predicate_meaning_idxs)






            #note CLS and SEP are discharder after Bi-LSTM, the Classifier takes in input only wokrds hidden state embedding
            arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
            list_arg_gt.append(arg_gt)
        

    list_arg_gt = torch.cat(list_arg_gt,dim = 0)
    list_pos_index = torch.cat(list_pos_index,dim = 0)
    list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
    list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
    list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
    gt["arg_gt"] = list_arg_gt
    input["predicate_index"] = list_predicate_index
    input["pos_index"] = list_pos_index.long()
    input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
    input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
    offset = batch_output.pop("offset_mapping")
    input["BERT_input"] = batch_output
    input["positional_encoding"] = list_positional_predicate_encoding.long()
    input["offset_mapping"] = offset
    input["gt"] = gt

   


    return input


















Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



model class is      : <class 'transformers.models.bert.modeling_bert.BertModel'>

model class is      : <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [19]:

#train_dataset = SRL("EN","train")

#note here we are directly loading args_roles mapping as computed before the the dasetet where we have perfomerd 
#EN and ES dataset should have the same consistency in generating 
train_dataset = SRL("ES",tokenizer,"train",train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)
#same mapping should be used in both the dataset
dev_dataset = SRL("ES",tokenizer,"dev",train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)

#### English-Spanish attempt

In [20]:
#note that parameter EN is only used for tracking on which dataset was/is trained, and activate loading of the pretrained head
#load the fine-tuned model over english
#path fine tuned
PATH_FINE = "hw2/stud/saved/model_2022_12_25_18_11_13Language_constained_training.pth"
model = Arg_Classifier(cfg)
model.load_state_dict(torch.load(PATH_FINE))
model.train().cuda()


Arg_Classifier(
  (bi_lstm_portable): LSTM(132, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (embedding_predicate_flag): Embedding(2, 32, max_norm=True)
  (embedding_predicate): Embedding(305, 50, max_norm=True)
  (embedding_pos): Embedding(19, 100, max_norm=True)
  (bi_lstm): LSTM(950, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout_language_constraint): Dropout(p=0.6, inplace=False)
  (dropout_in_classifier): Dropout(p=0.4, inplace=False)
  (Relu): ReLU()
  (Sigmoid): Sigmoid()
  (linear0): Linear(in_features=300, out_features=700, bias=True)
  (linear1): Linear(in_features=700, out_features=140, bias=True)
  (linear2): Linear(in_features=140, out_features=28, bias=True)
)

In [22]:
#check that it should be false
print(model.flag_dropout)
model.freeze_parts()

False
Freezed layer : bi_lstm_portable and embedding 


In [23]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime



currentDateAndTime = datetime.now()
_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)
_id = _id +"SP"



#optimizer = torch.optim.Adam(model.parameters(),lr = 0.000005)
optimizer = torch.optim.Adam(model.parameters())

scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()
auto_model.cuda()

EPOCHS = 200
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()

        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1)
        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    scheduler.step()

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    print("Epochs n.", epoch)
    print("F1 train:",f1)
    print("F1 avg train:",f1_avg)
    
    avg_train_loss = total_loss/counter
    writer.add_scalar("EN_Loss_ES/train", avg_train_loss, epoch)


    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("EN_Train_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("EN_Train_ES/classification", classification_result["f1"], epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------
    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
    

    writer.add_scalar("EN_Loss_ES/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("EN_Eval_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("EN_Eval_ES/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        print("SAVED :",PATH)
        torch.save(model.state_dict(),PATH)
    

Epochs n. 0
F1 train: [0.03809524 0.         0.         0.         0.         0.34298441
 0.         0.         0.40697674 0.         0.98162182 0.01526718
 0.36895161 0.         0.10218978 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
F1 avg train: 0.9482918882374491
identification {'true_positives': 790, 'false_positives': 264, 'false_negatives': 1339, 'precision': 0.7495256166982922, 'recall': 0.371066228276186, 'f1': 0.4963870562362551}
classification_result {'true_positives': 480, 'false_positives': 574, 'false_negatives': 1649, 'precision': 0.45540796963946867, 'recall': 0.22545796148426492, 'f1': 0.3016022620169651}
EPOCHS : 0
F1 eval : [0.         0.         0.         0.         0.         0.49646821
 0.         0.         0.55330136 0.         0.         0.98514152
 0.         0.45804196 0.         0.30208333 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
F1 avg eva

#### Compare without Transfert Learning 

In [25]:
model = Arg_Classifier(cfg).cuda()

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime



currentDateAndTime = datetime.now()
_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)+"WT"




#optimizer = torch.optim.Adam(model.parameters(),lr = 0.000005)
optimizer = torch.optim.Adam(model.parameters())

scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()
auto_model.cuda()

EPOCHS = 200
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1)
        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    scheduler.step()

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    print("Epochs n.", epoch)
    print("F1 train:",f1)
    print("F1 avg train:",f1_avg)
    
    avg_train_loss = total_loss/counter
    writer.add_scalar("Loss_ES/train", avg_train_loss, epoch)


    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Train_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("Train_ES/classification", classification_result["f1"], epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------
    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
    

    writer.add_scalar("Loss_ES/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Eval_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("Eval_ES/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        #print("SAVED :",PATH)
        #torch.save(model.state_dict(),PATH)
    

Epochs n. 0
F1 train: [0.         0.         0.         0.         0.         0.
 0.         0.         0.17486339 0.         0.97684792 0.
 0.0497076  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.        ]
F1 avg train: 0.9336283227735425
identification {'true_positives': 172, 'false_positives': 77, 'false_negatives': 1968, 'precision': 0.6907630522088354, 'recall': 0.08037383177570094, 'f1': 0.14399330263708668}
classification_result {'true_positives': 81, 'false_positives': 168, 'false_negatives': 2059, 'precision': 0.3253012048192771, 'recall': 0.03785046728971963, 'f1': 0.06781079949769779}
EPOCHS : 0
F1 eval : [0.         0.         0.         0.         0.         0.
 0.         0.         0.37042925 0.         0.         0.98274541
 0.         0.11848341 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
F1 avg eval : 0.946650

### French

#### New French Dataset
bert-base-multilingual-cased


In [26]:
import os
import json
import logging
import torch
from torch.utils.data import DataLoader,Dataset
import random
from typing import Dict

class SRL(Dataset):
 
    def __init__(self,language,tokenizer,path,args_roles = None,pos_list = None,predicate_dis = None) -> None:
        #train
        #self.path_root = 'data'
        #inference 
        self.path_root = 'hw2/stud/data'
        #self.path_root = 'stud/data'
        self.load_data(language,path)
        if args_roles is None :
            self.args_roles,self.list_broken_id = self.list_arg_roles()
            self.args_roles.append("UNK")
        else : 
            self.args_roles = args_roles
            _,self.list_broken_id = self.list_arg_roles()
        

        if pos_list is None :
            self.pos_list,_ = self.list_pos()
            self.pos_list.append("Nothing")
            self.pos_list.append("UNK")
        else : 
            self.pos_list = pos_list
        


        if predicate_dis is None :
            self.predicate_dis,_ = self.list_predicate_roles()
            self.predicate_dis.append("Nothing")
            self.predicate_dis.append("UNK")
        else : 
            self.predicate_dis = predicate_dis
        
        
        


        self.tokenizer = tokenizer

    def load_data(self,language,mode):
        
        mode = mode+".json"
        path = os.path.join(self.path_root,language,mode)
        data_file = open(path)
       
        data_ = json.load(data_file)

        list_data = []

        for data in data_:
            list_data.append(data_[data])
        

        self.data = list_data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, id : int):

        flag = False
        if id in self.list_broken_id :
            flag = True
            while flag == True:

                rand_id = random.randint(0, len(self.data)-1)
                
                if rand_id in self.list_broken_id :
                    pass
                else :
                    flag = False
                    id = rand_id        


        data = self.pre_processing(self.data[id])
        data = self.processig(data)
        return data
        
    def pre_processing(self, data:dict):
        data_list = []
        for role in data["roles"]:
            dictionary = dict()
            dictionary["words"] = data["words"]
            dictionary["role"] = data["roles"][role]
            dictionary["pre_idx"] = role
            dictionary["pos_tags"] = data["pos_tags"]
            dictionary["predicate_meaning"] = data["predicates"]
            data_list.append(dictionary)    
        return data_list
    
    def processig(self,data_list:list):
        
        for dictionary in data_list:

            #dictionary["words"] = data["words"]
            dictionary["gt_arg_identification"] = self.arg_id(dictionary["role"])
            dictionary["gt_arg_classification"] = self.arg_class(dictionary["role"])
            dictionary["pos_idx"] = self.pos_idx(dictionary["pos_tags"])
            dictionary["predicate_meaning_idx"] = self.predicate_meaning_idx(dictionary["predicate_meaning"])
        
        return data_list
   
    def list_arg_roles(self):
        list_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : roles = element["roles"]
            except : flag = False
            if flag :
                for e in roles:
                    sentence = element["roles"][e]

                    for word in sentence:
                        
                        list_roles.append(word)
                list_roles = list(set(list_roles))
            else : 
                list_broken_id.append(i)
        return list_roles,list_broken_id

    def list_predicate_roles(self):
        list_predicate_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : predicates = element["predicates"]
            except : flag = False
            if flag :
                for pre in predicates:
                    list_predicate_roles.append(pre)
                list_predicate_roles = list(set(list_predicate_roles))
            else : 
                list_broken_id.append(i)
        return list_predicate_roles,list_broken_id

    def list_pos(self):
        list_pos = []
        list_broken_id = []
        for i,element in enumerate(self.data):
            flag = True
            try : pos = element["pos_tags"]
            except : flag = False
            if flag :
                for e in pos:
                    list_pos.append(e)
                list_pos = list(set(list_pos))
            else : 
                list_broken_id.append(i)
        return list_pos,list_broken_id
  
    def arg_class(self,role:list):
        list_idxs = []
        for element in role:
            try : list_idxs.append(self.args_roles.index(element))
            except : list_idxs.append(self.args_roles.index("UNK"))
        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def arg_id(self,role:dict):
        list_idxs = []
        for element in role:
            if element == "_":
                list_idxs.append(0)
            else :
                list_idxs.append(1)

        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def pos_idx(self,pos_tags:dict):
        list_idxs = []
        list_idxs.append(self.pos_list.index("Nothing"))

        for element in pos_tags:
            try :list_idxs.append(self.pos_list.index(element))
            except :list_idxs.append(self.pos_list.index("UNK"))
        
        list_idxs.append(self.pos_list.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
    
    def predicate_meaning_idx(self,predicate_meaning_tags:dict):
        list_idxs = []
        list_idxs.append(self.predicate_dis.index("Nothing"))

        for element in predicate_meaning_tags:
            try : list_idxs.append(self.predicate_dis.index(element))
            except : list_idxs.append(self.predicate_dis.index("UNK"))
            
        
        list_idxs.append(self.predicate_dis.index("Nothing"))
        return torch.tensor(list_idxs, dtype=torch.int64)
   
    def role_gen(self,sentence):

        base = ["_"]*len(sentence["predicates"])
        roles_dict = dict()
        counter = 0
        for i,item in enumerate(sentence["predicates"]):

            if item != "_":
                base = ["_"]*len(sentence["predicates"])
                sentence["roles"] = 10
                roles_dict[str(i)] = base
                counter += 1
        
        if counter == 0:
            sentence["roles"] = { }
            flag = False
            
                
        else :
            sentence["roles"] = roles_dict
            flag = True

        return sentence,flag
        
    def prepare_batch(self,sentence):

        sentence,flag = self.role_gen(sentence)
        
        if flag :

            data = self.pre_processing(sentence)
            data = self.processig(data)
            data = [data]
            
            
            input = dict() 
            gt = dict()
            batch_sentence = [] 
            
            for period in data:
                for sentence in period :

                    
                
                    #print(len(sentence[0]["words"]))
                    pre_idx = int(sentence["pre_idx"])
                    

                    predicate = sentence["words"][pre_idx]

                    text = " ".join(sentence["words"])
                    tokens: list[str] = text.split()
                    predicate: list[str] = predicate.split()

                    #text = sentence[0]["words"]
                    
                    t = (tokens,predicate)

                    batch_sentence.append(t)
                
                
            
        
        

            batch_output = self.tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
            


            for period in data:

                list_positional_predicate_encoding = []
                list_predicate_index = [] 
                list_pos_index = [] 
                list_arg_gt = []
                list_predicate_meaning_index = []
                list_meaning_predicate_encoding = []

                for sentence in period:
                    #positional_encoding
                    #+2 per il CLS iniziale ad SEP finale
                    sentence_words_lenght =  len(sentence["words"])
                    positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    #+1 per il CLS iniziale
                    pre_idx = int(sentence["pre_idx"])
                    positional_predicate_encoding[:,pre_idx+1] = 1
                    list_positional_predicate_encoding.append(positional_predicate_encoding)
                    #print("positional_prefix_encoding",positional_predicate_encoding)
                    list_predicate_index.append(pre_idx)

                    meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
                    pre_idx = int(sentence["pre_idx"])
                    #rather then set the flag 0,1 set with class verb
                    meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
                    list_meaning_predicate_encoding.append(meaning_predicate_encoding)
                    

                    pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
                    list_pos_index.append(pos)
                    predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
                    list_predicate_meaning_index.append(predicate_meaning_idxs)


                    arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
                    list_arg_gt.append(arg_gt)


            list_arg_gt = torch.cat(list_arg_gt,dim = 0)
            list_pos_index = torch.cat(list_pos_index,dim = 0)
            list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
            list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
            list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
            gt["arg_gt"] = list_arg_gt
            input["predicate_index"] = list_predicate_index
            input["pos_index"] = list_pos_index.long()
            input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
            input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
            offset = batch_output.pop("offset_mapping")
            input["BERT_input"] = batch_output
            input["positional_encoding"] = list_positional_predicate_encoding.long()
            input["offset_mapping"] = offset
            input["gt"] = gt
        
        else :
            input = sentence






        return input,flag
    
# here we define our collate function
def collate_fn(batch) -> Dict[str, torch.Tensor]:
    #print(batch)
    input = dict() 
    batch_sentence = [] 
    #print(len(batch))
    for period in batch:
        for sentence in period :
        
            #print(len(sentence[0]["words"]))
            pre_idx = int(sentence["pre_idx"])
            

            predicate = sentence["words"][pre_idx]

            text = " ".join(sentence["words"])
            tokens: list[str] = text.split()
            predicate: list[str] = predicate.split()

            #text = sentence[0]["words"]
            
            t = (tokens,predicate)

            batch_sentence.append(t)
            #print(batch_sentence)

    batch_output = tokenizer.batch_encode_plus(batch_sentence,padding=True,is_split_into_words=True, truncation=True,return_offsets_mapping=True, return_tensors="pt")
    #print(batch_output.keys())


    gt = dict()
    
    


    for period in batch:

        list_positional_predicate_encoding = []
        list_arg_gt = []
        list_predicate_index = [] 
        list_pos_index = [] 
        list_predicate_meaning_index = []
        list_meaning_predicate_encoding = []

        for sentence in period:
            #positional_encoding
            #+2 per il CLS iniziale ad SEP finale
            sentence_words_lenght =  len(sentence["words"])
            positional_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            #+1 per il CLS iniziale
            pre_idx = int(sentence["pre_idx"])
            positional_predicate_encoding[:,pre_idx+1] = 1
            list_positional_predicate_encoding.append(positional_predicate_encoding)
            #print("positional_prefix_encoding",positional_predicate_encoding)
            list_predicate_index.append(pre_idx)


            meaning_predicate_encoding = torch.zeros(1,sentence_words_lenght+2)
            pre_idx = int(sentence["pre_idx"])
            #rather then set the flag 0,1 set with class verb
            meaning_predicate_encoding[:,pre_idx+1] = sentence["predicate_meaning_idx"][pre_idx+1]
            list_meaning_predicate_encoding.append(meaning_predicate_encoding)




            pos = torch.unsqueeze(sentence["pos_idx"],dim = 0)
            list_pos_index.append(pos)
            predicate_meaning_idxs = torch.unsqueeze(sentence["predicate_meaning_idx"],dim = 0)
            list_predicate_meaning_index.append(predicate_meaning_idxs)






            #note CLS and SEP are discharder after Bi-LSTM, the Classifier takes in input only wokrds hidden state embedding
            arg_gt = torch.unsqueeze(sentence["gt_arg_classification"],dim = 0)
            list_arg_gt.append(arg_gt)
        

    list_arg_gt = torch.cat(list_arg_gt,dim = 0)
    list_pos_index = torch.cat(list_pos_index,dim = 0)
    list_predicate_meaning_index = torch.cat(list_predicate_meaning_index,dim = 0)
    list_predicate_meaning_index_bis = torch.cat(list_meaning_predicate_encoding,dim = 0)
    list_positional_predicate_encoding = torch.cat(list_positional_predicate_encoding,dim = 0)
    gt["arg_gt"] = list_arg_gt
    input["predicate_index"] = list_predicate_index
    input["pos_index"] = list_pos_index.long()
    input["predicate_meaning_idx"] = list_predicate_meaning_index.long()
    input["predicate_meaning_idx_bis"] = list_predicate_meaning_index_bis.long()
    offset = batch_output.pop("offset_mapping")
    input["BERT_input"] = batch_output
    input["positional_encoding"] = list_positional_predicate_encoding.long()
    input["offset_mapping"] = offset
    input["gt"] = gt

   


    return input




In [28]:

#train_dataset = SRL("EN","train")

#note here we are directly loading args_roles mapping as computed before the the dasetet where we have perfomerd 
#EN and ES dataset should have the same consistency in generating 
train_dataset = SRL("FR",tokenizer,"train",train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)
#same mapping should be used in both the dataset
dev_dataset = SRL("FR",tokenizer,"dev",train_dataset.args_roles,train_dataset.pos_list,train_dataset.predicate_dis)

#### English-French attempt

In [33]:
#note that parameter EN is only used for tracking on which dataset was/is trained, and activate loading of the pretrained head
#load the fine-tuned model over english
model = Arg_Classifier(cfg)
model.load_state_dict(torch.load(PATH_FINE))
model.train().cuda()



Arg_Classifier(
  (bi_lstm_portable): LSTM(132, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (embedding_predicate_flag): Embedding(2, 32, max_norm=True)
  (embedding_predicate): Embedding(305, 50, max_norm=True)
  (embedding_pos): Embedding(19, 100, max_norm=True)
  (bi_lstm): LSTM(950, 50, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout_language_constraint): Dropout(p=0.6, inplace=False)
  (dropout_in_classifier): Dropout(p=0.4, inplace=False)
  (Relu): ReLU()
  (Sigmoid): Sigmoid()
  (linear0): Linear(in_features=300, out_features=700, bias=True)
  (linear1): Linear(in_features=700, out_features=140, bias=True)
  (linear2): Linear(in_features=140, out_features=28, bias=True)
)

#### Language constained training

In [34]:
model.freeze_parts()

Freezed layer : bi_lstm_portable and embedding 


In [32]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime



currentDateAndTime = datetime.now()
_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)




#optimizer = torch.optim.Adam(model.parameters(),lr = 0.000005)
optimizer = torch.optim.Adam(model.parameters())

scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()
auto_model.cuda()

EPOCHS = 200
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1)
        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    scheduler.step()

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    print("Epochs n.", epoch)
    print("F1 train:",f1)
    print("F1 avg train:",f1_avg)
    
    avg_train_loss = total_loss/counter
    writer.add_scalar("EN_Loss_ES/train", avg_train_loss, epoch)


    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("EN_Train_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("EN_Train_ES/classification", classification_result["f1"], epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------
    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
    

    writer.add_scalar("EN_Loss_ES/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("EN_Eval_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("EN_Eval_ES/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        print("SAVED :",PATH)
        torch.save(model.state_dict(),PATH)
    

Epochs n. 0
F1 train: [0.         0.0952381  0.         0.         0.         0.34222222
 0.         0.         0.43516874 0.         0.         0.98402296
 0.         0.41909385 0.         0.20408163 0.         0.
 0.         0.06451613 0.         0.         0.08163265 0.
 0.         0.         0.        ]
F1 avg train: 0.9544972524727422
identification {'true_positives': 961, 'false_positives': 285, 'false_negatives': 1324, 'precision': 0.7712680577849117, 'recall': 0.4205689277899344, 'f1': 0.5443217218918154}
classification_result {'true_positives': 601, 'false_positives': 645, 'false_negatives': 1684, 'precision': 0.4823434991974318, 'recall': 0.263019693654267, 'f1': 0.3404134806003965}
EPOCHS : 0
F1 eval : [0.         0.         0.         0.         0.         0.57250471
 0.         0.         0.52765058 0.         0.         0.98735714
 0.         0.4934688  0.         0.16066482 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.

FileNotFoundError: [Errno 2] No such file or directory: 'saved/model_2022_12_25_18_24_22.pth'

#### Compare without Transfert Learning 

In [ ]:
model = Arg_Classifier("FR",cfg).cuda()

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
from sklearn.metrics import f1_score,confusion_matrix
from torch.optim.lr_scheduler import ExponentialLR
from datetime import datetime


_id = str(currentDateAndTime.year)+"_"+str(currentDateAndTime.month)+"_"+str(currentDateAndTime.day)+"_"+str(currentDateAndTime.hour)+"_"+str(currentDateAndTime.minute)+"_"+str(currentDateAndTime.second)
_id =  _id +"FR"




#optimizer = torch.optim.Adam(model.parameters(),lr = 0.000005)
optimizer = torch.optim.Adam(model.parameters())

scheduler = ExponentialLR(optimizer, gamma=0.9)


logSotfMax = torch.nn.LogSoftmax(dim=1)
nll_loss = torch.nn.NLLLoss()

dataloader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

dataloader_dev = DataLoader(dev_dataset, batch_size=1, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)

mapping = dataloader_train.dataset.args_roles

auto_model.eval()
auto_model.cuda()

EPOCHS = 200
patience_counter = 0
patience = 5
max_val_loss = 9999
f1_score_max = 0
output_path = "hw2/stud/saved"
model_name = "model_"+_id+".pth"
PATH = os.path.join(output_path,model_name)


for epoch in range(EPOCHS):

    #TRAINING
    p = []
    g = []
    model.train()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_train):
        #print(sample_batched)

        
        optimizer.zero_grad()
       
        #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
        #------------------FILTERING SUB-WORDS----------------------
        subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
        word_emebedding = []
        for i in range(n):
            subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
            flag = subtoken_mask[0,i,0]
            if flag :
                continue
            else :
                word_emebedding.append(subwords_embedding)
        word_emebedding = torch.cat(word_emebedding,dim = 1)
        #-------------------------FORWARD/BACKWARD----------------------------------
        x = model.forward(subwords_embeddings = output_hidden_states_sum,
            perdicate_positional_encoding = sample_batched["positional_encoding"],
            predicate_index = sample_batched["predicate_index"],
            pos_index_encoding = sample_batched["pos_index"],
            predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])        
        b,n = sample_batched["gt"]["arg_gt"].size()
        loss = nll_loss(logSotfMax(x),gt)
        total_loss = total_loss + loss
        #print(loss)
        loss.backward()
        optimizer.step()
    

        counter += 1 
            

        #-------------------------RESULT STORING----------------------------------
        predicted = torch.argmax(x, dim=1)
        p += predicted.tolist()
        g += gt.tolist()
    

    #-------------------------RESULTS----------------------------------
    scheduler.step()

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    print("Epochs n.", epoch)
    print("F1 train:",f1)
    print("F1 avg train:",f1_avg)
    
    avg_train_loss = total_loss/counter
    writer.add_scalar("Loss_ES/train", avg_train_loss, epoch)


    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Train_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("Train_ES/classification", classification_result["f1"], epoch)



    #EVALUATION
    p = []
    g = []
    model.eval()
    total_loss = 0
    counter = 0
    for i_batch, sample_batched in enumerate(dataloader_dev):
    
      #----------------------PREPARE INPUT/OUTPUT-------------------------------
        input_bert = sample_batched["BERT_input"]
        input_bert['input_ids'] = input_bert['input_ids'].cuda()
        input_bert['token_type_ids'] = input_bert['token_type_ids'].cuda()
        input_bert['attention_mask'] = input_bert['attention_mask'].cuda()
        sample_batched["positional_encoding"] = sample_batched["positional_encoding"].cuda()
        sample_batched["pos_index"] = sample_batched["pos_index"].cuda()
        #sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx"].cuda()
        sample_batched["predicate_meaning_idx"] = sample_batched["predicate_meaning_idx_bis"].cuda()
        #prepare gt
        gt = torch.flatten(sample_batched["gt"]["arg_gt"]).cuda()
        offset = sample_batched["offset_mapping"]
        #-----------------BERT EMBEDDING---------------------------
        with torch.no_grad():
            output = auto_model(**input_bert)
            output_hidden_states_sum = torch.stack(output.hidden_states[-4:], dim=0).sum(dim=0)
            b,n,h = output_hidden_states_sum.size()
    
            #------------------FILTERING SUB-WORDS----------------------
            subtoken_mask = torch.unsqueeze(offset[:,:, 0] != 0,dim =-1)
            word_emebedding = []
            for i in range(n): 
                subwords_embedding = torch.unsqueeze(output_hidden_states_sum[:,i,:],dim = 1)
                flag = subtoken_mask[0,i,0]
                if flag :
                    continue
                else :
                    word_emebedding.append(subwords_embedding)
            word_emebedding = torch.cat(word_emebedding,dim = 1)
            #-------------------------FORWARD----------------------------------
            x = model.forward(subwords_embeddings = output_hidden_states_sum,
                        perdicate_positional_encoding = sample_batched["positional_encoding"],
                        predicate_index = sample_batched["predicate_index"],
                        pos_index_encoding = sample_batched["pos_index"],
                        predicate_meaning_encoding = sample_batched["predicate_meaning_idx"])   


            b,n = sample_batched["gt"]["arg_gt"].size()
            loss = nll_loss(logSotfMax(x),gt)
            total_loss = total_loss + loss
            #-------------------------RESULT STORING----------------------------------
            predicted = torch.argmax(x, dim=1)
            p += predicted.tolist()
            g += gt.tolist()
            counter += 1 
    
    #-------------------------RESULTS----------------------------------
    avg_eval_loss = total_loss/counter

    if avg_eval_loss < max_val_loss:
        max_val_loss = avg_eval_loss
    else :
        patience_counter += 1
    

    f1 = f1_score(g, p, average=None)
    f1_avg = f1_score(g, p, average="weighted")

    if patience_counter >= patience :  


        print("Early stopping at epoch : ",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
        break
    else :
        print("EPOCHS :",epoch)
        print("F1 eval :",f1)
        print("F1 avg eval :",f1_avg)
    

    writer.add_scalar("Loss_ES/validation", avg_eval_loss, epoch)

    g,p = mapping_args(g,p,mapping)

    identification_result,classification_result = metrics(g,p)
    print("identification",identification_result)
    print("classification_result",classification_result)

    writer.add_scalar("Eval_ES/identification", identification_result["f1"], epoch)
    writer.add_scalar("Eval_ES/classification", classification_result["f1"], epoch)




    if f1_avg > f1_score_max:
        f1_score_max = f1_avg
        #print("SAVED :",PATH)
        #torch.save(model.state_dict(),PATH)
    

Epochs n. 0
F1 train: [0.         0.19946809 0.         0.9768764  0.         0.0295421
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
F1 avg train: 0.9336015019908596
identification {'true_positives': 172, 'false_positives': 71, 'false_negatives': 1987, 'precision': 0.7078189300411523, 'recall': 0.07966651227420102, 'f1': 0.14321398834304747}
classification_result {'true_positives': 85, 'false_positives': 158, 'false_negatives': 2074, 'precision': 0.3497942386831276, 'recall': 0.03937007874015748, 'f1': 0.070774354704413}
EPOCHS : 0
F1 eval : [0.         0.23416618 0.         0.9777718  0.00265252 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
F1 avg eval : 0.9389493973815146
ide